# HASH CODE BOOKS AND LIBRARIES 

l'obbiettivo è ottimizzare la scansione dei libri.

Ogni libro verra assegnato ad un libreria e avrà i seguenti attributi:
B= numeri di libri differenti
ID = da 0 a B-1

### Vincolo 1 :
scansionare ogni libro una sola volta

 L= numero di librerie
 SB = set libri in ogni libreria
 T = tempo per iscrivere alla libreria
 N = numero di libri scansionati/day

### Vincolo 2 :
solo una libreria alla volta puo essere iscritta

## SOLUTION

1. LETTURA INPUT
2. Calcola un punteggio euristico per ogni libreria con questa formula:

   
   $$ Libray Score = (LibraryBookScore)/(Library SignUp Days) $$


   dove library_books_score è calcolato in questo modo:

   1. Calcola i giorni disponibili per questa libreria:

   $$ Libray Avaible Days = (All Days) - (Library Signup Days)  $$


   2. Conta quanti libri possono essere scansionati in questi giorni:

  
   $$ K = (Libray Avaible Days)*(LibraryMaximumBooksScannedPerDay)  $$


   3. Ottieni i primi k libri in base al loro punteggio e somma i loro punteggi. Questo è il *library_books_score*.

3. Ordina le librerie in base al loro punteggio
4. Per ogni libreria in ordine:
   1. Conta quanti libri di questa libreria (k) possono essere scansionati, come nei passaggi 2i e 2ii, ma ora prendi in considerazione il numero di giorni di registrazione dalle librerie precedenti. Quindi, se le librerie precedenti hanno impiegato *d* giorni totali per l'iscrizione, la formula del passaggio 2i diventa:
   

   $$ Libray Avaible Days = (All Days)  -  (Library Signup Days)  -  D $$
 
   2. Dai libri di questa biblioteca, ad eccezione dei libri che sono già stati scansionati dalle biblioteche precedenti, prendi i primi k libri in base al loro punteggio e scansionali.

5. CREA l'output


In [5]:
import numpy as np
import operator
import sys

def ordinamento(array_1, array_2):
    for i in range(len(array_1)):
        for j in range(len(array_2)):
            #print(array_1)
            if array_1[i]>array_1[j]:
                #print("scambio")
                array_1[i], array_1[j]= array_1[j], array_1[i]
                array_2[i], array_2[j]= array_2[j], array_2[i]
    return array_1 , array_2



##### CLASSE LIBRARY

###### ATTRIBUTI :
    ID : id libreria
    BOOK_IDS : insieme id dei libri
    SIGNUP_DAYS : giorno della registrazione
    MAX_BOOK_SCANNED_PER_DAY : quantita massima di libri/giorno

###### VARIABILI GLOBALI :
    BOOK SCORE :
    LIBRARIES_SIGNUP_DAYS :
    LIBRARIES_MAX_BOOKS_SCANNED_PER_DAY :
    D :
    FINAL_BOOK :

###### METODI :
    GET_BEST_BOOKS_IDS :
    GET_BEST_BOOKS_SCORE :
    SUM_BOOK_SCORE :

In [6]:
class Biblioteca:

#iniziallizzo
    def __init__(self, id, id_libri, registrazione, scanned_su_day):
        self.id = id
        self.id_libri = id_libri
        self.registrazione = registrazione
        self.scanned_su_day = scanned_su_day

#definisco il miglior libro
    def top_libri_id(self, giorno=0):
        global punteggio_libri, registrazione, biblioteche_max_books_scanned_su_day, D, libri_totali
        
        giorni_disponibili = D - self.registrazione - giorno
        
        #elimino i libri da quelli disponibili
        #Set è una raccolta non ordinata, non modificabile* e non indicizzata. Nessun membro duplicato.
        libri_disponibili = list(set(self.id_libri) - set(libri_totali))
        #prende lo score solamente dei libri disponibili
        punteggio_attuale = np.take(punteggio_libri, libri_disponibili)

        #giorni disponibili*num_max_book/day per ogni oggetto e prendo il max
        max_num_libri_scan = max(min(int(giorni_disponibili * self.scanned_su_day), len(libri_disponibili)), 0)

        if max_num_libri_scan == 0:
            return []

        # get top k books (k=max_num_libri_scan)
        ind = np.argpartition(punteggio_attuale, -max_num_libri_scan)[-max_num_libri_scan:]

        #ritorna i libri disponibili per ID
        return np.take(libri_disponibili, ind)

#definisco il miglior score
    def top_punteggio_libri(self, giorno=0):

          # DEF VARIABILI
        global punteggio_libri, registrazione, biblioteche_max_books_scanned_su_day, D
        giorni_disponibili = D - self.registrazione - giorno

        # NUMERO MASSIMO DI LIBRI SCANSIONABILI
        max_num_libri_scan = max(min(int(giorni_disponibili * self.scanned_su_day), len(self.id_libri)), 0)

        #ritorna un array contenente gli score solamente dei libri presenti nella libreria
        punteggio_attuale = np.take(punteggio_libri, self.id_libri)

        # get top k books (k=max_num_libri_scan)
        #ritorna gli id da 0 a max num books dei libri con lo score maggiore
        ind = np.argpartition(punteggio_attuale, -max_num_libri_scan)[-max_num_libri_scan:]
        
        best_books_scores = np.take(punteggio_attuale, ind)

        return np.sum(best_books_scores)

    def __repr__(self):
        return self.id.__str__()

In [7]:
#sommo gli score dei libri
def somma_punteggio_libri(id_libri):
    global punteggio_libri
    return np.sum(np.take(punteggio_libri, list(id_libri)))

files = ["a_example", "b_read_on", "c_incunabula", "d_tough_choices", "e_so_many_books", "f_libraries_of_the_world"]

totale_punteggio = 0

def swap(lista, gruppi1 , unita1 , setoff1):
    
    gruppi=int(gruppi1)
    unita=int(unita1)

    setoff=int(setoff1)
    
    lista_modify=list(lista)

    for i in range(gruppi):
        max=i*setoff+unita
        if max<len(lista_modify):
            for j in range(unita):
                tmp = lista_modify[i*setoff] 
                lista_modify[i*setoff] =lista_modify[i*setoff+j]
                lista_modify[i*setoff+j]=tmp
    lista2=tuple(lista_modify)
    return lista2

In [8]:

total_score=0
score=0

for file in files:

## LETTURA DA FILE
    with open("inputs/" + file + ".txt", "r") as f:
        content = f.read().splitlines()
    print(file)

    libri_esistenti, tot_biblio, D = list(map(int, content[0].split(' ')))

    punteggio_libri = list(map(int, content[1].split(' ')))
    pos = 1
    numero_libri = np.zeros(tot_biblio)
    registrazione = np.zeros(tot_biblio)
    biblioteche_scanned_su_day = np.zeros(tot_biblio)
    biblioteche = np.empty(tot_biblio, dtype=Biblioteca)
    for i in range(tot_biblio):
        pos += 1
        n, t, m = list(map(int, content[pos].split(' ')))
        numero_libri[i] = n
        registrazione[i] = t
        biblioteche_scanned_su_day[i] = m
        
        pos += 1
        id_libri = np.asarray(list(map(int, content[pos].split(' '))))
        biblioteche[i] = Biblioteca(i, id_libri, t, m)

## INIZIO PROGRAMMA
    sys.stdout.write("\rSolving...")
    
    heuristic_score = np.vectorize(lambda A, B: A/B)
    punteggio_totale = heuristic_score(numero_libri, registrazione)
    
    punteggio_totale, biblioteche_ordinate = ordinamento(punteggio_totale, biblioteche)
    Gruppi=len(biblioteche)/30
    unita_gruppo=len(biblioteche)/Gruppi/1.5
    setoff=unita_gruppo*3

    biblioteche_ordinate_2=swap(biblioteche_ordinate,Gruppi,unita_gruppo,setoff)

### SALVATAGGIO FILE
    libri_totali = set()
    with open("outputs/" + file + ".out", 'w+') as f:
        
        f.write(str(tot_biblio) + "\n")

        giorno = 0
        for i in range(tot_biblio):
            if giorno<=D:
                biblioteca_attuale = biblioteche_ordinate_2[i]
                chosen_id_libri = biblioteca_attuale.top_libri_id(giorno)
                libri_totali.update(chosen_id_libri)
                giorno += biblioteca_attuale.registrazione

                if len(chosen_id_libri) > 0:
                    f.write(str(biblioteca_attuale.id) + " " + str(len(chosen_id_libri)) + "\n")
                    f.write(str(' '.join(map(str, chosen_id_libri))) + "\n")
                else:
                    f.write(str(biblioteca_attuale.id) + " 1\n")
                    f.write(str(biblioteca_attuale.id_libri[0])+"\n")
                c=i+1
                progress = 100 * (i+1) / (tot_biblio)
                sys.stdout.write("\r Caricamento                        (" + str(int(progress)) + " %)")

        punteggio = somma_punteggio_libri(libri_totali)
        totale_punteggio += punteggio

        print("\r-  Punteggio:        ", punteggio)
        print("                totale biblioteche iscritte:        ",  c  , " su  " , tot_biblio)

print("")
print("totale punteggio:", totale_punteggio)



a_example
-  Punteggio:         21            (100 %)
                totale biblioteche iscritte:         2  su   2
b_read_on
-  Punteggio:         5815700       (91 %)
                totale biblioteche iscritte:         91  su   100
c_incunabula
-  Punteggio:         5586053       (13 %)
                totale biblioteche iscritte:         1310  su   10000
d_tough_choices
-  Punteggio:         4813445       (50 %)
                totale biblioteche iscritte:         15001  su   30000
e_so_many_books
-  Punteggio:         4600802       (13 %)
                totale biblioteche iscritte:         133  su   1000
f_libraries_of_the_world
-  Punteggio:         5224557       (1 %)
                totale biblioteche iscritte:         17  su   1000

totale punteggio: 26040578
